# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re

# Crawler

In [73]:
Dominio = ['https://www.capefeargames.com/','https://www.mtgotraders.com/',
           'https://www.wizardscupboard.com/','https://abugames.com/',
           'https://www.cardkingdom.com/','http://www.starcitygames.com/',
           'https://www.tcgplayer.com/','https://scryfall.com/']

paginasVisitadas = []

## * Baseline

In [74]:
# Função para marcar a page como visitada.
def PaginaVisitda (url):
    if url not in paginasVisitadas:
        #paginasVisitdas.append(url)
        return False
    else:
        return True
    
    return False

In [77]:
def CardsCrawler(url):
    #request
    codigo_fonte = rq.get(url)
    
    # Verificando o Content-type
    cabeçalho = codigo_fonte.headers.get('content-type')
    if 'text/html' in cabeçalho:
        print('Arquivo HTML!')

In [82]:
count = 1
for url in Dominio:
    CardsCrawler(url)
    print(count)
    count+=1

Arquivo HTML!
1
Arquivo HTML!
2
Arquivo HTML!
3
Arquivo HTML!
4
Arquivo HTML!
5
Arquivo HTML!
6
Arquivo HTML!
7
Arquivo HTML!
8


## * Heurística

# Classificação

# Extração